# Webscraping

I have some data that I want to pull from a [webpage](https://en.wikipedia.org/wiki/Piano_key_frequencies), and I don't want to do it by hand. So I'm getting python to do it.

In [65]:
from requests import get
from bs4 import BeautifulSoup
from csv import writer as csv_writer
from re import compile as re_compile

Get the webpage

In [13]:
r = get('https://en.wikipedia.org/wiki/Piano_key_frequencies')
page = BeautifulSoup(r.content, 'html.parser')
print(page.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Piano key frequencies - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"cacff7a4-fd08-461c-8e49-6ad85a1a073a","wgCSPNonce":false,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Piano_key_frequencies","wgTitle":"Piano key frequencies","wgCurRevisionId":1050784290,"wgRevisionId":1050784290,"wgArticleId":1325784,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Short description matches Wikidata","Articles needing additional references f

I'm going to use Chrome's devtools to locate the table that I want to extract.

There seems to be one table with the class "wikitable" so I can just grab that.

We can also query the number of rows in the table body.

We only need the first 88 so we can filter those.

Apparently, all of the tables data will be contained in the body, and Wikipedia uses some javascript to move it to the head, so we'll filter those rows manually

In [18]:
table = page.find('table', 'wikitable')
data_row = lambda elem: elem.name == 'tr' and not elem.find('th')
html_rows = table.find_all(data_row)
html_rows

[<tr bgcolor="lightgray">
 <td>108
 </td>
 <td>119
 </td>
 <td>b′′′′′</td>
 <td>B<sub>8</sub></td>
 <td>7902.133</td>
 <td>
 </td>
 <td>
 </td>
 <td>
 </td>
 <td>
 </td>
 <td>
 </td>
 <td>
 </td></tr>,
 <tr>
 <td bgcolor="darkslategray" style="color:white">107
 </td>
 <td bgcolor="lightgray">118
 </td>
 <td bgcolor="lightgray">a<span class="music-symbol" style="font-family: Arial Unicode MS, Lucida Sans Unicode;"><span class="music-sharp">♯</span></span>′′′′′/b<span class="music-symbol" style="font-family: Arial Unicode MS, Lucida Sans Unicode;"><span class="music-flat">♭</span></span>′′′′′
 </td>
 <td bgcolor="lightgray">A<span class="music-symbol" style="font-family: Arial Unicode MS, Lucida Sans Unicode;"><span class="music-sharp">♯</span></span><sub>8</sub>/B<span class="music-symbol" style="font-family: Arial Unicode MS, Lucida Sans Unicode;"><span class="music-flat">♭</span></span><sub>8</sub>
 </td>
 <td bgcolor="lightgray">7458.620
 </td>
 <td bgcolor="lightgray">
 </td>
 <td b

Great! Now we can extract the data from these rows

In [68]:
# We are only interested in a select set of columns
cols = 0, 3, 4

# Get all 'td' cells in rows, filter select columns, and get text from cells
notes = [ row.find_all('td') for row in html_rows ]
notes = [ [ row[c] for c in cols ] for row in notes ]
notes = [ [ cell.get_text().strip() for cell in row ] for row in notes ]
notes

[['108', 'B8', '7902.133'],
 ['107', 'A♯8/B♭8', '7458.620'],
 ['106', 'A8', '7040.000'],
 ['105', 'G♯8/A♭8', '6644.875'],
 ['104', 'G8', '6271.927'],
 ['103', 'F♯8/G♭8', '5919.911'],
 ['102', 'F8', '5587.652'],
 ['101', 'E8', '5274.041'],
 ['100', 'D♯8/E♭8', '4978.032'],
 ['99', 'D8', '4698.636'],
 ['98', 'C♯8/D♭8', '4434.922'],
 ['88', 'C8 Eighth octave', '4186.009'],
 ['87', 'B7', '3951.066'],
 ['86', 'A♯7/B♭7', '3729.310'],
 ['85', 'A7', '3520.000'],
 ['84', 'G♯7/A♭7', '3322.438'],
 ['83', 'G7', '3135.963'],
 ['82', 'F♯7/G♭7', '2959.955'],
 ['81', 'F7', '2793.826'],
 ['80', 'E7', '2637.020'],
 ['79', 'D♯7/E♭7', '2489.016'],
 ['78', 'D7', '2349.318'],
 ['77', 'C♯7/D♭7', '2217.461'],
 ['76', 'C7 Double high C', '2093.005'],
 ['75', 'B6', '1975.533'],
 ['74', 'A♯6/B♭6', '1864.655'],
 ['73', 'A6', '1760.000'],
 ['72', 'G♯6/A♭6', '1661.219'],
 ['71', 'G6', '1567.982'],
 ['70', 'F♯6/G♭6', '1479.978'],
 ['69', 'F6', '1396.913'],
 ['68', 'E6', '1318.510'],
 ['67', 'D♯6/E♭6', '1244.508'],
 [

Now I just want to clean up the data before writing it. 

First I only want the exact note from the note (minus the Pedal C, Double Pedal C, etc), which I will do with regular expressions.

I also want to sort the notes by their key number. I can convert the key number to an int and sort by key number.

Finally, after this, I only want the first 88 keys to fit the standard keyboard.

In [70]:
# Note regex
note = re_compile(r'[A-G](♯|♭)?[0-9](/[A-G](♯|♭)[0-9])?')
extract = lambda s: note.search(s).group()

# Notes to test regex
test_notes = 0, 1, 11
print_test = False
if print_test:
    for i in test_notes:
        print(extract(notes[i][1]))

# Convert notes (and key numbers)
# Sort by key number
# Take first 88
notes = [[ int(row[0]), extract(row[1]), row[2] ] for row in notes ]
notes.sort(key=lambda row: row[0])
notes = notes[:88]
notes

[[1, 'A0', '27.50000'],
 [2, 'A♯0/B♭0', '29.13524'],
 [3, 'B0', '30.86771'],
 [4, 'C1', '32.70320'],
 [5, 'C♯1/D♭1', '34.64783'],
 [6, 'D1', '36.70810'],
 [7, 'D♯1/E♭1', '38.89087'],
 [8, 'E1', '41.20344'],
 [9, 'F1', '43.65353'],
 [10, 'F♯1/G♭1', '46.24930'],
 [11, 'G1', '48.99943'],
 [12, 'G♯1/A♭1', '51.91309'],
 [13, 'A1', '55.00000'],
 [14, 'A♯1/B♭1', '58.27047'],
 [15, 'B1', '61.73541'],
 [16, 'C2', '65.40639'],
 [17, 'C♯2/D♭2', '69.29566'],
 [18, 'D2', '73.41619'],
 [19, 'D♯2/E♭2', '77.78175'],
 [20, 'E2', '82.40689'],
 [21, 'F2', '87.30706'],
 [22, 'F♯2/G♭2', '92.49861'],
 [23, 'G2', '97.99886'],
 [24, 'G♯2/A♭2', '103.8262'],
 [25, 'A2', '110.0000'],
 [26, 'A♯2/B♭2', '116.5409'],
 [27, 'B2', '123.4708'],
 [28, 'C3', '130.8128'],
 [29, 'C♯3/D♭3', '138.5913'],
 [30, 'D3', '146.8324'],
 [31, 'D♯3/E♭3', '155.5635'],
 [32, 'E3', '164.8138'],
 [33, 'F3', '174.6141'],
 [34, 'F♯3/G♭3', '184.9972'],
 [35, 'G3', '195.9977'],
 [36, 'G♯3/A♭3', '207.6523'],
 [37, 'A3', '220.0000'],
 [38, 'A♯

Great! Now we can write this data to a CSV file (for use in my virtual synth notebook)

In [71]:
with open('data/notes.csv', 'w+') as f:
    csv = csv_writer(f)
    csv.writerows(data)